In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from sentence_transformers import SentenceTransformer, util

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import json

/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `l

### Preparing dataset
1. user_id
2. review_text
3. plot_synopsis
4. review_text embeddings
5. cosine similarity
6. ngram vector
7. review text length
   

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
directory_path = "/SFS/project/ry/dp_sgteam/catherine/ada/dataset"
df_reviews = pd.read_json(f"{directory_path}/cleaned_data.json")
df_reviews['label'] = df_reviews['is_spoiler'].astype(int)

df_movies = pd.read_json(f"{directory_path}/IMDB_movie_details.json", lines=True)

print(f"Dataset shape: {df_reviews.shape}")
print(f"Dataset shape: {df_movies.shape}")
df_reviews.head()

Dataset shape: (573913, 8)
Dataset shape: (1572, 7)


,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary,label
0,10 February 2006,tt0111161,ur1898687,1,oscar year shawshank redemption write direct f...,10,A classic piece of unforgettable film-making.,1
1,6 September 2000,tt0111161,ur0842118,1,shawshank redemption without doubt one brillia...,10,Simply amazing. The best film of the 90's.,1
2,3 August 2001,tt0111161,ur1285640,1,believe film best story ever tell film tell ti...,8,The best story ever told on film,1
3,1 September 2002,tt0111161,ur1003471,1,yes spoiler film emotional impact find hard wr...,10,Busy dying or busy living?,1
4,20 May 2004,tt0111161,ur0226855,1,heart extraordinary movie brilliant indelible ...,8,"Great story, wondrously told and acted",1


In [4]:
df = pd.merge(df_reviews, df_movies, on='movie_id', how='left')

print("Data loaded and merged successfully.")
df['label'] = df['is_spoiler'].astype(int)
df['user_id'] = df['user_id'].str[2:].astype(int)

df = df[['user_id', 'review_text', 'plot_synopsis', 'label']]
df['review_text_length'] = df['review_text'].str.len()

df.head()

Data loaded and merged successfully.


,user_id,review_text,plot_synopsis,label,review_text_length
0,1898687,oscar year shawshank redemption write direct f...,"In 1947, Andy Dufresne (Tim Robbins), a banker...",1,2776
1,842118,shawshank redemption without doubt one brillia...,"In 1947, Andy Dufresne (Tim Robbins), a banker...",1,709
2,1285640,believe film best story ever tell film tell ti...,"In 1947, Andy Dufresne (Tim Robbins), a banker...",1,962
3,1003471,yes spoiler film emotional impact find hard wr...,"In 1947, Andy Dufresne (Tim Robbins), a banker...",1,2148
4,226855,heart extraordinary movie brilliant indelible ...,"In 1947, Andy Dufresne (Tim Robbins), a banker...",1,2735


In [5]:
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)
print(f"Data split into train ({len(train_df)}) and test ({len(test_df)}) sets.")

Data split into train (459130) and test (114783) sets.


In [6]:
for obj in (train_df, test_df):
    obj.reset_index(drop=True, inplace=True)

In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

HTTP Error 500 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
HTTP Error 500 thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 2s [Retry 2/5].


In [8]:
def embedd_to_vector(text):
    print("Encoding texts...")
    embeddings = model.encode(text.fillna('').tolist(), 
                              convert_to_tensor=True, show_progress_bar=True, device=device)
    print("Encoded.")
    return embeddings.cpu().numpy().tolist()

def concat_to_df(main_df, embedd_list, column_name):
    embedd_df = pd.DataFrame(embedd_list)
    num_dims = len(embedd_df.columns)
    new_names = [f'{column_name}_{i}' for i in range(num_dims)]
    embedd_df.columns = new_names
    df = pd.concat([main_df, embedd_df], axis = 1)
    print(f"Finish concatenate: with dataframe {len(main_df)} and list {len(embedd_df)}")
    return df

In [9]:
train_review_embeddings = embedd_to_vector(train_df['review_text'])
train_synopsis_embeddings = embedd_to_vector(train_df['plot_synopsis'])

train_df = concat_to_df(train_df, train_review_embeddings, "dim")

Encoding texts...


Batches: 100%|████████████████████████████████████████████████████████████████| 14348/14348 [01:38<00:00, 146.15it/s]


Encoded.
Encoding texts...


Batches: 100%|█████████████████████████████████████████████████████████████████| 14348/14348 [08:54<00:00, 26.86it/s]


Encoded.
Finish concatenate: with dataframe 459130 and list 459130


In [10]:
test_review_embeddings = embedd_to_vector(test_df['review_text'])
test_synopsis_embeddings = embedd_to_vector(test_df['plot_synopsis'])

test_df = concat_to_df(test_df, test_review_embeddings, "dim")

Encoding texts...


Batches: 100%|███████████████████████████████████████████████████████████████████| 3587/3587 [00:44<00:00, 80.98it/s]


Encoded.
Encoding texts...


Batches: 100%|███████████████████████████████████████████████████████████████████| 3587/3587 [02:18<00:00, 25.83it/s]


Encoded.
Finish concatenate: with dataframe 114783 and list 114783


In [11]:
print("Calculating similarities for training set...")
sim_scores_train = util.pairwise_cos_sim(train_review_embeddings, train_synopsis_embeddings)
train_df['sim_score_synopsis_review'] = sim_scores_train.flatten()

print("Calculating similarities for testing set...")
sim_scores_test = util.pairwise_cos_sim(test_review_embeddings, test_synopsis_embeddings)
test_df['sim_score_synopsis_review'] = sim_scores_test.flatten()

Calculating similarities for training set...
Calculating similarities for testing set...


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_features = 500,
    stop_words = 'english',
    ngram_range=(1,2)
)

train_tfidf = vectorizer.fit_transform(train_df['review_text'])
train_tfidf_df = concat_to_df(train_df, train_tfidf, "tfidf")

test_tfidf = vectorizer.transform(test_df['review_text'])
test_tfidf_df = concat_to_df(test_df, test_tfidf, "tfidf")

print(f"Shape of TF-IDF matrices: Train = {train_tfidf_df.shape}, Test = {test_tfidf_df.shape}")

Finish concatenate: with dataframe 459130 and list 459130
Finish concatenate: with dataframe 114783 and list 114783


NameError: name 'X_train_tfidf' is not defined

In [29]:
train_tfidf_df = concat_to_df(train_df, train_tfidf.toarray().tolist(), "tfidf")
test_tfidf_df = concat_to_df(test_df, test_tfidf.toarray().tolist(), "tfidf")

Finish concatenate: with dataframe 459130 and list 459130
Finish concatenate: with dataframe 114783 and list 114783


### Storing the dataset

In [ ]:
train_df.to_json(f"{directory_path}/train_data.json",  orient="records", indent=4)
test_df.to_json(f"{directory_path}/test_data.json",  orient="records", indent=4)

### Model

In [30]:
train_xg = train_tfidf_df.drop(columns= ['review_text', 'plot_synopsis'])
test_xg = test_tfidf_df.drop(columns= ['review_text', 'plot_synopsis'])
train_xg.head()

,user_id,label,review_text_length,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,...,tfidf_490,tfidf_491,tfidf_492,tfidf_493,tfidf_494,tfidf_495,tfidf_496,tfidf_497,tfidf_498,tfidf_499
0,5291991,1,1821,-0.005757,-0.051729,-0.021550,-0.005745,-0.074771,0.078937,-0.069202,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.077083,0.0,0.000000,0.0
1,48053412,1,829,-0.002652,-0.056541,0.058831,-0.003095,0.043479,0.030117,0.054024,...,0.0,0.116315,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
2,28438054,1,829,-0.017446,-0.091489,0.014513,-0.028227,-0.026655,0.060746,0.043278,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
3,35553121,0,182,-0.081114,0.030047,0.042871,0.093194,-0.001539,-0.004182,0.031489,...,0.0,0.000000,0.0,0.223546,0.0,0.0,0.000000,0.0,0.000000,0.0
4,14069613,0,1333,-0.051064,-0.072159,-0.044826,-0.012800,-0.006169,0.065184,0.030398,...,0.0,0.000000,0.0,0.077469,0.0,0.0,0.131571,0.0,0.097174,0.0


In [31]:
from sklearn.decomposition import TruncatedSVD

X_train = train_xg.drop(columns=["label"])
y_train = train_xg["label"]

X_test = test_xg.drop(columns=["label"])
y_test = test_xg["label"]

In [32]:
n = y_train.value_counts()
pos_weight = round(n[0]/n[1],2)

print(f"Positive Weight: {pos_weight}")

Positive Weight: 2.8


In [33]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from xgboost import XGBClassifier

clf = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    min_child_weight=1,
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",          # use "gpu_hist" if you have a GPU
    random_state=42,
    n_jobs=-1,
    scale_pos_weight = 2.8
)

clf.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)

# Evaluation
proba = clf.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)
print("AUC:", roc_auc_score(y_test, proba))
print("Accuracy:", accuracy_score(y_test, pred))
print("F1:", f1_score(y_test, pred))


AUC: 0.7936331037915815
Accuracy: 0.718486186978908
F1: 0.5663557672951754


In [34]:
print(classification_report(y_test, pred, target_names = ['Spoiler', 'Not Spoiler']))

              precision    recall  f1-score   support

     Spoiler       0.87      0.73      0.79     84598
 Not Spoiler       0.48      0.70      0.57     30185

    accuracy                           0.72    114783
   macro avg       0.67      0.71      0.68    114783
weighted avg       0.77      0.72      0.73    114783



In [39]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, classification_report

clf = XGBClassifier(
    objective="binary:logistic",
    eval_metric="auc",
    tree_method="hist",          # Use "gpu_hist" if you have a GPU
    random_state=42,
    n_jobs=-1,
    scale_pos_weight=pos_weight
)

param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [4, 6, 8],
    'subsample': [0.7, 0.8],
    'colsample_bytree': [0.7, 0.8]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring='recall',
    cv=cv,
    verbose=2,
    n_jobs=-1
)

print("Starting GridSearchCV...")
grid_search.fit(X_train, y_train)

# 5. Analyze the Results
print("\n--- GridSearchCV Results ---")
print("Best Parameters Found: ", grid_search.best_params_)
print("Best Recall Score (on cross-validation): ", grid_search.best_score_)

best_clf = grid_search.best_estimator_

print("\n--- Evaluation on Test Set using Best Estimator ---")
proba = best_clf.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

print("AUC:", roc_auc_score(y_test, proba))
print("Accuracy:", accuracy_score(y_test, pred))
print("F1 Score:", f1_score(y_test, pred))
print("Recall:", recall_score(y_test, pred)) # Most important metric for this search

print("\nClassification Report on Test Set:")
print(classification_report(y_test, pred))

Starting GridSearchCV...
Fitting 3 folds for each of 108 candidates, totalling 324 fits


/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages

[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.7; total time= 6.7min
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=4, n_estimators=300, subsample=0.8; total time=15.9min


/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.7; total time= 6.5min
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=4, n_estimators=300, subsample=0.8; total time=16.0min
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.8; total time= 6.7min
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=4, n_estimators=300, subsample=0.8; total time=15.9min
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=4, n_estimators=100, subsample=0.8; total time= 6.5min
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=4, n_estimators=300, subsample=0.7; total time=16.1min


...

[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=4, n_estimators=300, subsample=0.7; total time= 5.3min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=4, n_estimators=300, subsample=0.8; total time= 5.6min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=100, subsample=0.7; total time= 3.0min


/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=4, n_estimators=500, subsample=0.7; total time= 8.5min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=100, subsample=0.8; total time= 3.1min


/home/kangcat/miniconda3/envs/documentation/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


[CV] END colsample_bytree=0.8, learning_rate=0.05, max_depth=8, n_estimators=500, subsample=0.8; total time=14.9min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=4, n_estimators=500, subsample=0.8; total time= 8.6min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=300, subsample=0.7; total time= 7.2min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=6, n_estimators=500, subsample=0.8; total time=10.6min
[CV] END colsample_bytree=0.8, learning_rate=0.1, max_depth=8, n_estimators=300, subsample=0.8; total time= 9.3min
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=8, n_estimators=100, subsample=0.7; total time= 8.6min
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=8, n_estimators=100, subsample=0.8; total time= 6.7min
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=8, n_estimators=300, subsample=0.7; total time=12.6min
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=8, n_estimators=500

In [40]:
importances = pd.Series(best_clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print("\nTop 20 Feature Importances (from best model):")
print(importances.head(20))


Top 20 Feature Importances (from best model):
tfidf_421                    0.057258
review_text_length           0.054565
dim_319                      0.049397
dim_223                      0.032518
user_id                      0.027520
tfidf_241                    0.016684
dim_49                       0.010562
tfidf_109                    0.010258
tfidf_98                     0.009354
tfidf_129                    0.009290
sim_score_synopsis_review    0.008263
dim_187                      0.007698
tfidf_326                    0.007459
dim_127                      0.006769
dim_244                      0.006760
dim_298                      0.006752
tfidf_198                    0.006263
dim_92                       0.005889
dim_132                      0.005788
dim_263                      0.005462
dtype: float32


In [42]:
print(classification_report(y_test, pred, target_names=['Not Spoiler', 'Spoiler']))

              precision    recall  f1-score   support

 Not Spoiler       0.87      0.67      0.76     84598
     Spoiler       0.44      0.72      0.54     30185

    accuracy                           0.68    114783
   macro avg       0.65      0.69      0.65    114783
weighted avg       0.76      0.68      0.70    114783

